# Pre-process input data for coastal variable extraction

Author: Emily Sturdivant; esturdivant@usgs.gov

***

Pre-process files to be used in extractor.ipynb (Extract barrier island metrics along transects). See the project [README](https://github.com/esturdivant-usgs/BI-geomorph-extraction/blob/master/README.md) and the Methods Report (Zeigler et al., in review). 


## Pre-processing steps

1. Pre-created geomorphic features: dunes, shoreline points, armoring.
2. Inlets
3. Shoreline
4. Transects - extend and sort
5. Transects - tidy


## Notes:
This process requires some manipulation of the spatial layers by the user. When applicable, instructions are described in this file.

***

## Import modules

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import arcpy
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
try:
    import core.functions_warcpy as fwa
    import core.functions as fun
except ImportError as err:
    print("Looks like you need to install the module to your ArcGIS environment. Please see the README for details.")
    
from core.setvars import *

No module named 'CoastalVarExtractor'
Looks like you need to install the module to your ArcGIS environment.
To do so: pip install git+https://github.com/esturdivant-usgs/BI-geomorph-extraction.git


ImportError: No module named 'CoastalVarExtractor'

If you don't want to formally install the module, you'll need to add the path to the package to your system path: 

```python
mod_path = r"path\to\dir\BI-geomorph-extraction" # replace with path to module
sys.path.insert(0, mod_path)
import CoastalVarExtractor.functions_warcpy as fwa
```

## Initialize variables

Based on the project directory, and the site and year you have input, setvars.py will set a bunch of variables as the names of folders, files, and fields. 1) set-up the project folder and paths: 

In [3]:
from core.setvars import *

# Inputs - vector
orig_trans = os.path.join(arcpy.env.workspace, 'DelmarvaS_SVA_LT')
ShorelinePts = os.path.join(home, 'SLpts')
dlPts = os.path.join(home, 'DLpts')
dhPts = os.path.join(home, 'DHpts')
# Inputs - raster
elevGrid = os.path.join(home, 'DEM')
elevGrid_5m = os.path.join(home, 'DEM_5m')
SubType = os.path.join(home, 'FI11_SubType')
VegType = os.path.join(home, 'FI11_VegType')
VegDens = os.path.join(home, 'FI11_VegDens')
GeoSet = os.path.join(home, 'FI11_GeoSet')
DisMOSH = os.path.join(home, 'FI11_DisMOSH')

# Files to create or modify
armorLines = os.path.join(home, 'armorLines')
inletLines = os.path.join(home, 'inletLines')
SA_bounds = 'SA_bounds'

# Outputs
extendedTrans = os.path.join(home, 'extTrans')
extTrans_tidy = os.path.join(home, 'tidyTrans')
barrierBoundary = os.path.join(home, 'bndpoly_2sl')  
shoreline = os.path.join(home, 'ShoreBetweenInlets')

tr_w_anthro = os.path.join(home, 'extTrans_wAnthro')

## Dunes and armoring <a name="geofeatures"></a>
Display the points and the DEM in a GIS to check for irregularities. For example, if shoreline points representing a distance less than X m are visually offset from the general shoreline, they should likely be removed. Another red flag is when the positions of dlows and dhighs in relation to the shore are illogical, i.e. dune crests are seaward of dune toes. 

If fill values in the morphology datasets are not -99999, then replace them will Null values. If they are -99999, the extractor can accept fill values as long as they match those in the rest of the extractor. It also accepts Null (None or np.nan) values. 

The morphology datasets do not need to be reprojected to UTM because the find_ClosestPt2Trans_snap() function will reproject them if necessary. 

#### Replace fill values with Null. 
Only necessary if the fill values are different from what will be used during the extraction routine to follow (default is -99999).

In [7]:
fwa.ReplaceValueInFC(dhPts, oldvalue=fill, newvalue=None, fields=["dhigh_z"])      # Dhighs
fwa.ReplaceValueInFC(dlPts, oldvalue=fill, newvalue=None, fields=["dlow_z"])       # Dlows
fwa.ReplaceValueInFC(ShorelinePts, oldvalue=fill, newvalue=None, fields=["slope"]) # Shoreline

'\\\\Mac\\stor\\Projects\\TransectExtraction\\FireIsland2010\\FireIsland2010.gdb\\SLpts'

#### Project to UTM if not already. 
If this happens, we need to change the file name for future processing. 

#### If desired, delete dune points with missing Z values. 
Not necessary because you can choose to exclude those points from the beach width calculation. 

In [ ]:
# Delete points with fill elevation value from dune crests
fmapdict = fwa.find_similar_fields('DH', dhPts, fields=['_z'])
arcpy.CopyFeatures_management(dhPts, dhPts+'_orig')
fwa.DeleteFeaturesByValue(dhPts, [fmapdict['_z']['src']], deletevalue=-99999)
print('Deleted dune crest points that will fill elevation values. Original file is saved with the _orig suffix.')

# Delete points with fill elevation value from dune toes
fmapdict = fwa.find_similar_fields('DL', dlPts, fields=['_z'])
arcpy.CopyFeatures_management(dlPts, dlPts+'_orig')
fwa.DeleteFeaturesByValue(dlPts, [fmapdict['_z']['src']], deletevalue=-99999)
print('Deleted dune toe points that will fill elevation values. Original file is saved with the _orig suffix.')

#### Armoring
If the dlows do not capture the entire top-of-beach due to atypical formations caused by anthropogenic modification, you may need to digitize the beachfront armoring. The next code block will generate an empty feature class. Refer to the DEM and orthoimagery. If there is no armoring in the study area, continue. If there is armoring, use the Editing toolbar to add lines to the feature class that trace instances of armoring. Common manifestations of what we call armoring are sandfencing and sandbagging and concrete seawalls. 

If there is no armoring file in the project geodatabase, the extractor script will notify you that it is proceeding without armoring.

*__Requires manipulation in GIS__*

In [ ]:
arcpy.CreateFeatureclass_management(home, os.path.basename(armorLines), 'POLYLINE', spatial_reference=utmSR)
print("{} created. Now manually digitize the shorefront armoring.".format(armorLines))

## Inlets
We also need to manually digitize inlets if an inlet delineation does not already exist. To do, the code below will produce the feature class. After which, use the Editing toolbar to create a line where the oceanside shore meets a tidal inlet. If the study area includes both sides of an inlet, that inlet will be represented by two lines. The inlet lines are use to define the bounds of the oceanside shore, which is also considered the point where the oceanside shore meets the bayside. Inlet lines must intersect the MHW contour. 

What do we do when the study area and not an inlet is the end?

*__Requires manipulation in GIS__*

In [ ]:
# manually create lines that correspond to end of land and cross the MHW line (use bndpoly/DEM)
arcpy.CreateFeatureclass_management(home, os.path.basename(inletLines), 'POLYLINE', spatial_reference=utmSR)
print("{} created. Now we'll stop for you to manually create lines at each inlet.".format(inletLines))

## Shoreline
The shoreline is produced through a combination of the DEM and the shoreline points. The first step converts the DEM to both MTL and MHW contour polygons. Those polygons are combined to produce the full shoreline, which is considered to fall at MHW on the oceanside and MTL on the bayside (to include partially submerged wetland).

If the study area does not end cleanly at an inlet, create a separate polyline feature class (default name is 'SA_bounds') and add lines that bisect the shoreline; they should look and function like inlet lines. Specify this in the arguments for DEMtoFullShorelinePoly() and CreateShoreBetweenInlets().

At some small inlets, channel depth may be above MTL. In this case, the script left to its own devices will leave the MTL contour between the two inlet lines. This can be rectified after processing by deleting the mid-inlet features from the temp file 'shore_2split.' 

In [10]:
SA_bounds = 'SA_bounds'

bndpoly = fwa.DEMtoFullShorelinePoly(elevGrid_5m, sitevals['MTL'], sitevals['MHW'], inletLines, ShorelinePts)
print('Select features from {} that should not be included in the final shoreline polygon. '.format(bndpoly))

Creating the MTL contour polgon from the DEM...
Creating the MHW contour polgon from the DEM...
Combining the two polygons...
Isolating the above-MTL portion of the polygon to the bayside...

User input required! Select extra features in bndpoly for deletion.

        Recommended technique: select the polygon/s to keep and then Switch Selection.



NameError: name 'barrierBoundary' is not defined

*__Requires display in GIS__*

User input is required to identify only the areas within the study area and eliminate isolated landmasses that are not. Once the features to delete are selected, either delete in the GIS or run the code below. Make sure the bndpoly variable matches the layer name in the GIS.
__Do not...__ select the features in ArcGIS and then run DeleteFeatures in this Notebook Python kernel. That will delete the entire feature class. 

```
arcpy.DeleteFeatures_management(bndpoly)
```

The next step snaps the boundary polygon to the shoreline points anywhere they don't already match and as long as as they are within 25 m of each other. 

In [12]:
bndpoly = 'bndpoly'
barrierBoundary = fwa.NewBNDpoly(bndpoly, ShorelinePts, barrierBoundary, '25 METERS', '50 METERS')

Created: \\Mac\stor\Projects\TransectExtraction\FireIsland2010\FireIsland2010.gdb\bndpoly_2sl


In [5]:
shoreline = fwa.CreateShoreBetweenInlets(barrierBoundary, inletLines, shoreline, 
                                         ShorelinePts, proj_code)

Splitting \\Mac\stor\Projects\TransectExtraction\FireIsland2010\FireIsland2010.gdb\bndpoly_2sl_edited at inlets...
Preserving only those line segments that intersect shoreline points...
Dissolving the line to create \\Mac\stor\Projects\TransectExtraction\FireIsland2010\FireIsland2010.gdb\ShoreBetweenInlets...


After this step, you'll want to make sure the shoreline looks okay. There should be only one line segment for each stretch of shore between two inlets. Segments may be incorrectly deleted if the shoreline points are missing in the area. Segments may be incorrectly preserved if they are intersect a shoreline point. To rectify, either perform manual editing or rerun this code with modifications. 

## Transects - extend, sort, and tidy

Create extendedTrans, NASC transects for the study area extended to cover the island, with gaps filled, and sorted in the field sort_ID.

### 1. Extend the transects and use a copy of the lines to fill alongshore gaps

In [7]:
# Delete transects over 200 m outside of the study area.
if input("Need to remove extra transects? 'y' if barrierBoundary should be used to select. ") == 'y':
    fwa.RemoveTransectsOutsideBounds(orig_trans, barrierBoundary)
trans_extended = fwa.ExtendLine(orig_trans, os.path.join(arcpy.env.scratchGDB, 'trans_ext_temp'), extendlength, proj_code)
trans_presort = fwa.CopyAndWipeFC(trans_extended, os.path.join(arcpy.env.scratchGDB, 'trans_presort_temp'), ['sort_ID'])
print("MANUALLY: use groups of existing transects in new FC '{}' to fill gaps.".format(trans_presort))

Need to remove extra transects? 'y' if barrierBoundary exists and should be used to select. y
\\Mac\stor\Projects\TransectExtraction\Fisherman2014\Fisherman2014.gdb\DelmarvaS_SVA_LT is already projected in UTM.
MANUALLY: use groups of existing transects in new FC '\\Mac\stor\Projects\TransectExtraction\Fisherman2014\scratch.gdb\trans_presort_temp' to fill gaps.


*__Requires manipulation in GIS__*

1. Edit the trans_presort_temp feature class. __Move and rotate__ groups of transects to fill in gaps that are greater than 50 m alongshore. There is no need to preserve the original transects, but avoid overlapping the transects with each other and with the originals. Do not move any transects slightly. If they are moved, they will not be deleted in the next stage. If you slightly move any, you can either undo or delete that line entirely.

In [8]:
fwa.RemoveDuplicates(trans_presort, trans_extended, barrierBoundary)

'\\\\Mac\\stor\\Projects\\TransectExtraction\\Fisherman2014\\scratch.gdb\\trans_presort_temp'

### 2. Sort the transects along the shore
Usually if the shoreline curves, we need to identify different groups of transects for sorting. This is because the GIS will not correctly establish the alongshore order by simple ordering from the identified sort_corner. If this is the case, answer __yes__ to the next prompt.

In [6]:
sort_lines = fwa.SortTransectPrep(spatialref=utmSR)

Do we need to sort the transects in batches to preserve the order? (y/n) y
MANUALLY: Add features to sort_lines. Indicate the order of use in 'sort' and the sort corner in 'sort_corn'.


*__Requires manipulation in GIS__*

The last step generated an empty sort lines feature class if you indicated that transects need to be sorted in batches to preserve the order. Now, the user creates lines that will be used to spatially sort transects in groups. 

For each group of transects:

1. __Create a new line__ in 'sort_lines' that intersects all transects in the group. The transects intersected by the line will be sorted independently before being appended to the preceding groups.  (*__add example figure__*)
2. __Assign values__ for the fields 'sort,' 'sort_corner,' and 'reverse.' 'sort' indicates the order in which the line should be used and 'sort_corn' indicates the corner from which to perform the spatial sort ('LL', 'UL', etc.). 'reverse' indicates whether the order should be reversed (roughly equivalent to 'DESCENDING').
3. Run the following code to create a new sorted transect file.


In [2]:
fwa.SortTransectsFromSortLines(trans_presort, extendedTrans, sort_lines, tID_fld)

Creating new feature class \\Mac\stor\Projects\TransectExtraction\Fisherman2014\Fisherman2014.gdb\extTrans to hold sorted transects...
Sorting sort lines by field sort...
For each line, creating subset of transects and adding them in order to the new FC...
Copying the generated OID values to the transect ID field (sort_ID)...


<Result '\\\\Mac\\stor\\Projects\\TransectExtraction\\Fisherman2014\\Fisherman2014.gdb\\extTrans'>

### 3. Tidy the extended (and sorted) transects to remove overlap

*__Requires manipulation in GIS__*

Overlapping transects cause problems during conversion to 5-m points and to rasters. We create a separate feature class with the 'tidied' transects, in which the lines don't overlap. This is largely a manually process with the following steps: 

1. __Select__ transects to be used to split other transects. Prioritize transects that a) were originally from NASC, b) have dune points within 25 m, and c) are oriented perpendicular to shore. (*__add example figure__*)
2. Use the __Copy Features__ geoprocessing tool to copy only the selected transects into a new feature class. If desired, here is the code that could be used to copy the selected features and clear the selection:
    ```python
    arcpy.CopyFeatures_management(extendedTrans, overlapTrans_lines)
    arcpy.SelectLayerByAttribute_management(extendedTrans, "CLEAR_SELECTION")
    ```
3. Run the code below to split the transects at the selected lines of overlap.

In [8]:
overlapTrans_lines = os.path.join(arcpy.env.scratchGDB, 'overlapTrans_lines_temp')
if not arcpy.Exists(overlapTrans_lines):
    overlapTrans_lines = input("Filename of the feature class of only 'boundary' transects: ")
trans_x = arcpy.Intersect_analysis([extendedTrans, overlapTrans_lines], 
                                   os.path.join(arcpy.env.scratchGDB, 'overlap_points_temp'),
                                   'ALL', output_type="POINT")
arcpy.SplitLineAtPoint_management(extendedTrans, trans_x, extTrans_tidy)

<Result '\\\\Mac\\stor\\Projects\\TransectExtraction\\Fisherman2014\\Fisherman2014.gdb\\tidyTrans3'>

Delete the extraneous segments __manually__. Recommended:

1. Using __Select with Line__ draw a line to the appropriate side of the boundary transects. This will select the line segments that need to be deleted. 
2. __Delete__ the selected lines.
3. Remove any remaining overlaps entirely by hand. Use the __Split Line__ tool in the Editing toolbar to split lines to be shortened at the points of overlap. Then delete the remnant sections. 

# Run the extractor!